In [1]:
import json

from openai_service import get_chat_completion
from prompts import *
import pandas as pd
import json
import os
from tqdm.auto import tqdm
import random
import string
tqdm.pandas()

/Users/abhishekanand/miniforge3/envs/focus-llm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
def process_row(row, answer_key, processed_data):
    id = row['id']
    if id not in processed_data:
        prompt = row['prompt']
        text_response, prompt_tokens, completion_tokens, total_tokens = get_chat_completion(prompt=prompt, model="gpt-4-0125-preview")
        processed_data[id] = {
            'response': text_response,
            'prompt_tokens': prompt_tokens,
            'completion_tokens': completion_tokens,
            'total_tokens': total_tokens,
            'correct_answer': row[answer_key]
        }
        return True
    return False

# ARC

In [23]:
arc_df = pd.read_csv('datasets/arc_challenge_train_with_prompt.csv')
arc_df.head()

,id,question,choices,answerKey,options,prompt
0,Mercury_SC_415702,George wants to warm his hands quickly by rubb...,"{'text': array(['dry palms', 'wet palms', 'pal...",A,"['dry palms', 'wet palms', 'palms covered with...",Question: - George wants to warm his hands qui...
1,MCAS_2009_5_6516,Which of the following statements best explain...,{'text': array(['The refrigerator door is smoo...,B,"['The refrigerator door is smooth.', 'The refr...",Question: - Which of the following statements ...
2,Mercury_7233695,A fold observed in layers of sedimentary rock ...,"{'text': array(['cooling of flowing magma.', '...",B,"['cooling of flowing magma.', 'converging of c...",Question: - A fold observed in layers of sedim...
3,Mercury_7041615,Which of these do scientists offer as the most...,"{'text': array(['worldwide disease', 'global m...",D,"['worldwide disease', 'global mountain buildin...",Question: - Which of these do scientists offer...
4,Mercury_7041860,A boat is acted on by a river current flowing ...,"{'text': array(['west', 'east', 'north', 'sout...",B,"['west', 'east', 'north', 'south']",Question: - A boat is acted on by a river curr...


In [24]:
len(arc_df), len(arc_df['id'].unique())

(1119, 1119)

In [129]:
PROCESSED_DATA_PATH = "./datasets/arc_challenge_train_gpt4_response.json"

In [130]:
processed_data = {}

In [131]:
if os.path.exists(PROCESSED_DATA_PATH):
    with open(PROCESSED_DATA_PATH, 'r') as file:
        processed_data = json.load(file)

In [132]:
len(processed_data)

500

In [134]:
processed_status = arc_df.iloc[500:1000].progress_apply(lambda row: process_row(row, answer_key='answerKey', processed_data=processed_data), axis=1)

  0%|          | 0/500 [00:00<?, ?it/s]

100%|██████████| 500/500 [2:48:47<00:00, 20.26s/it]  


In [135]:
len(processed_data)

1000

In [136]:
with open(PROCESSED_DATA_PATH, 'w') as file:
	json.dump(processed_data, file)

In [137]:
len(processed_status[processed_status])

500

# Winogrande

In [19]:
wino_df = pd.read_csv('datasets/winogrande_train_with_prompt.csv')
wino_df.head()

,question,option1,option2,answer,prompt,id
0,John moved the couch from the garage to the ba...,garage,backyard,1,Question: - John moved the couch from the gara...,GO3wkLp9TC
1,The doctor diagnosed Justin with bipolar and R...,Justin,Robert,2,Question: - The doctor diagnosed Justin with b...,IPQZOkoLLz
2,Dennis drew up a business proposal to present ...,Dennis,Logan,1,Question: - Dennis drew up a business proposal...,W85KDUn8gA
3,Felicia unexpectedly made fried eggs for break...,Felicia,Katrina,2,Question: - Felicia unexpectedly made fried eg...,UIUCGI68OY
4,My shampoo did not lather easily on my Afro ha...,shampoo,hair,2,Question: - My shampoo did not lather easily o...,WFAUCgvz91


In [22]:
wino_df_sampled = wino_df.sample(n=990).reset_index(drop=True)
wino_df_sampled.head()

,question,option1,option2,answer,prompt,id
0,Laura gave a class on drawing to a group inclu...,Laura,Kayla,2,Question: - Laura gave a class on drawing to a...,NcYXNaT5zN
1,John cleaned the table with his bare hand and ...,table,water,2,Question: - John cleaned the table with his ba...,HkUfjZSeAx
2,So _ was embarrassed because Mary wears a wig ...,Mary,Kayla,1,Question: - So _ was embarrassed because Mary ...,z79MTrfX9f
3,I like working on the old operating systems ov...,new operating systems,old operating systems,2,Question: - I like working on the old operatin...,33wGXmn6z5
4,"When visiting Canadian National Parks, I purch...",Annual Park Pass,Daily Park Pass,1,Question: - When visiting Canadian National Pa...,OLcn9DE9IC


In [18]:
# wino_df['id'] = [''.join(random.choices(string.ascii_letters + string.digits, k=10)) for _ in range(len(wino_df))]
# wino_df.head()

In [57]:
WINO_PROCESSED_DATA_PATH = "./datasets/winogrande_train_gpt4_response.json"

processed_data = {}
if os.path.exists(WINO_PROCESSED_DATA_PATH):
    with open(WINO_PROCESSED_DATA_PATH, 'r') as file:
        processed_data = json.load(file)
len(processed_data)

1000

In [53]:
processed_status = wino_df[wino_df['id'] == 'G1iDf6VN4H'].progress_apply(lambda row: process_row(row, answer_key='answer', processed_data=processed_data), axis=1)

100%|██████████| 1/1 [00:17<00:00, 17.96s/it]


In [54]:
with open(WINO_PROCESSED_DATA_PATH, 'w') as file:
	json.dump(processed_data, file)

In [55]:
len(processed_status[processed_status])

1